In [4]:
import re
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json

objects = []
articles = [
    ['cbt', "https://en.wikipedia.org/wiki/Cognitive_behavioral_therapy"],
    ['vr', "https://en.wikipedia.org/wiki/Virtual_reality"],
    ['ai', "https://en.wikipedia.org/wiki/Artificial_intelligence"]
]
for [topic, url] in articles:
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    paras = []
    for para in soup.find(id="mw-content-text").find_all('p'):
        para = para.get_text()
        para = para.replace('\n', ' ')
        para = re.sub(r"\[[0-9]+\]", "", para)
        if len(para) < 10:
            continue
        # para = re.sub(r"\[[0-9]*\]", "", para)
        if len(paras) == 0:
            paras.append(para)
        elif len(para) < 100:
            paras[len(paras) - 1] += para
        else:
            paras.append(para)
    for i, para in enumerate(paras):
        doc = dict(
            text=para,
            title=f"{topic}{i}"
        )
        objects.append(doc)

with open("mock_entries.json", "w") as f:
    f.write(json.dumps(objects))

In [1]:
USE_GPU = False
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import json

from haystack.document_stores.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore.load('./index.faiss', './index.config')

In [2]:
from haystack.nodes import DensePassageRetriever
dpr_retriever = DensePassageRetriever(
    document_store=document_store,
    use_gpu=False
    # query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    # passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [3]:
from haystack.nodes import RAGenerator
rag_generator = RAGenerator(
    model_name_or_path="facebook/rag-sequence-nq",
    retriever=dpr_retriever,
    top_k=3,
    min_length=3
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [4]:
from haystack.pipelines import GenerativeQAPipeline

from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)
pipe = ExtractiveQAPipeline(reader, dpr_retriever)
result = pipe.run(query='What is CBT?', params={"Retriever": {"top_k": 5}})
a = 1

# # Change `minimum` to `medium` or `all` to raise the level of detail
# print_answers(result, details="all")
print(result)
return None

Inferencing Samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.02 Batches/s]

{'query': 'What is CBT?', 'no_ans_gap': 11.73118257522583, 'answers': [<Answer {'answer': 'Brief cognitive behavioral therapy', 'type': 'extractive', 'score': 0.9323877692222595, 'context': 'Brief cognitive behavioral therapy (BCBT) is a form of CBT which has been developed for situations in which there are time constraints on the therapy ', 'offsets_in_document': [{'start': 0, 'end': 34}], 'offsets_in_context': [{'start': 0, 'end': 34}], 'document_id': '68bec10980b1ec8d80e70569070c437d', 'meta': {'name': 'cbt81', 'obj_id': '1ff6313f-127b-47f3-b888-545b584efd77', 'parent_id': '450497e3-d704-4558-b607-4eb0d214f39a', 'obj_type': 'entry', 'vector_id': '115'}}>, <Answer {'answer': 'Cognitive emotional behavioral therapy', 'type': 'extractive', 'score': 0.7439917922019958, 'context': 'Cognitive emotional behavioral therapy (CEBT) is a form of CBT developed initially for individuals with eating disorders but now used with a range of ', 'offsets_in_document': [{'start': 0, 'end': 38}], 'offse

SyntaxError: 'return' outside function (2443452032.py, line 13)